In [2]:
# Neural net stuff
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import activations

# number manip
import numpy as np
import pandas as pd
import math
import random as rand

# plotting
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# I also want to hide stinky warning boxes, and 
# this: https://stackoverflow.com/questions/9031783/hide-all-warnings-in-ipython
# had some solutions.
def hide_warnings():
    from IPython.display import HTML
    HTML('''<script>
    var code_show_err = false; 
    var code_toggle_err = function() {
     var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
     var stderr = Array.from(stderrNodes)
     if (code_show_err){
         stderr.forEach(ele => ele.style.display = 'block');
     } else {
         stderr.forEach(ele => ele.style.display = 'none');
     }
     code_show_err = !code_show_err
    } 
    document.addEventListener('DOMContentLoaded', code_toggle_err);
    </script>
    To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')
    
hide_warnings()
###########################################################
print(f"Tensor Flow Version: {tf.__version__}")


Tensor Flow Version: 2.4.0
